1.네이버에서 "코스피"라는 키워드를 입력
<input id="query" name="query" type="search" title="검색어를 입력해 주세요." placeholder="검색어를 입력해 주세요." maxlength="255" autocomplete="off" class="search_input" data-atcmp-element="">

2.검색버튼 클릭
<button type="submit" class="btn_search" onclick="window.ntm.push({event:&quot;nclick&quot;,el:this,click_area:&quot;sch.action&quot;})"> <span class="ico_btn_search_svg"> <svg id="search-btn" viewBox="0 0 50 50" xmlns="http://www.w3.org/2000/svg" style="fill: #03c75a">  </svg> </span> <span class="blind">검색</span> </button>

3.뉴스탭 클릭
<a role="tab" href="?ssc=tab.news.all&amp;where=news&amp;sm=tab_jum&amp;query=%EC%BD%94%EC%8A%A4%ED%94%BC" onclick="return goOtherCR(this,'a=tab*n.jmp&amp;r=8&amp;i=&amp;u='+urlencode(this.href));" class="tab" aria-selected="false"><i class="spnew2 ico_nav_news"></i>뉴스</a>

4.옵션버튼 활성화
<a href="javascript:;" role="button" aria-expanded="false" aria-haspopup="listbox" class="btn_option _search_option_open_btn" aria-label="검색옵션 열기" onclick="tCR('a=opt.unfd');">옵션<i class="spnew ico_filter_arr"></i></a>

5.기간 직접입력 활성화
<a href="#" role="tab" class="txt txt_option _calendar_select_trigger " data-select-trigger="" aria-selected="false" onclick="tCR('a=opt.cal');">직접입력<i class="spnew ico_check">옵션<em class="open">펼치기</em><em class="close">접기</em></i></a>

6.시작일 설정(20160101)
<a href="#" role="tab" class="spnew_bf ico_calendar _start_trigger selected" aria-selected="true" onclick="tCR('a=opt.str');">2025.04.02.</a>

7.종료일 설정(20160131)
<a href="#" role="tab" class="spnew_bf ico_calendar _end_trigger" onclick="tCR('a=opt.fin');" aria-selected="true">2025.04.02.</a>

8.기간적용버튼 클릭
<button type="button" class="btn_apply _apply_btn">적용</button>

9.네이버뉴스라는 a태그가 있는 기사들만 제목이랑 URL 수집하기
<a href="https://n.news.naver.com/mnews/article/001/0015305069?sid=101" class="info" target="_blank" onclick="return goOtherCR(this, 'a=nws*a.nav&amp;r=1&amp;i=880000D8_000000000000000015305069&amp;u='+urlencode(this.href));">네이버뉴스</a>

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [12]:
# 날짜 선택 함수
def select_dropdown_date(driver, year: str, month: str, day: str, is_start=True):
    trigger_class = "_start_trigger" if is_start else "_end_trigger"
    driver.find_element(By.CLASS_NAME, trigger_class).click()
    time.sleep(0.5)

    # 연도 선택
    year_xpath = f'//strong[text()="년(Year)"]/following-sibling::div//li[@data-value="{year}"]/a'
    driver.find_element(By.XPATH, year_xpath).click()
    time.sleep(0.2)

    # 월 선택
    month_xpath = f'//strong[text()="월(Month)"]/following-sibling::div//li[@data-value="{int(month)}"]/a'
    driver.find_element(By.XPATH, month_xpath).click()
    time.sleep(0.2)

    # 일 선택
    day_xpath = f'//strong[text()="일(Day)"]/following-sibling::div//li[@data-value="{int(day)}"]/a'
    driver.find_element(By.XPATH, day_xpath).click()
    time.sleep(0.5)

In [18]:
keyword = "코스피"
start_date = ("2016", "2", "1")
end_date = ("2016", "2", "28")

In [20]:
# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)


driver.get("https://www.naver.com")
time.sleep(1)

# 검색창에 입력
driver.find_element(By.ID, "query").send_keys(keyword)
driver.find_element(By.CLASS_NAME, "btn_search").click()
time.sleep(2)

# 뉴스 탭 클릭
driver.find_element(By.XPATH, '//a[contains(@href, "where=news") and contains(@class, "tab")]').click()
time.sleep(2)

# 옵션 열기
driver.find_element(By.CLASS_NAME, "_search_option_open_btn").click()
time.sleep(1)

# 직접입력 선택
driver.find_element(By.CLASS_NAME, "_calendar_select_trigger").click()
time.sleep(1)

# 날짜 입력
select_dropdown_date(driver, *start_date, is_start=True)
select_dropdown_date(driver, *end_date, is_start=False)

# 적용 클릭
driver.find_element(By.CLASS_NAME, "_apply_btn").click()
time.sleep(2)

# 스크롤 다운
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 네이버뉴스만 수집
results = []
naver_links = driver.find_elements(By.XPATH, '//a[text()="네이버뉴스"]')

for link in naver_links:
    try:
        article_block = link.find_element(By.XPATH, "../../..")
        title_tag = article_block.find_element(By.CSS_SELECTOR, "a.news_tit")
        title = title_tag.text
        url = link.get_attribute("href")
        # 날짜 추출
        date_span = article_block.find_element(By.CSS_SELECTOR, "span.info")
        date = date_span.text
        results.append({"title": title, "url": url, "date": date})
    except Exception as e:
        continue

# # 출력
# for r in results:
#     print(f"[제목] {r['title']}\n[URL] {r['url']}\n")

df = pd.DataFrame(results)
df.to_csv("../data/코스피_201602.csv", index=False, encoding='utf-8-sig')
print("CSV 파일로 저장 완료")

driver.quit()

CSV 파일로 저장 완료
